code to create whole-brain maps and fdr-correction for each of the 4 neural codes

In [ ]:
import pdb
import copy
import numpy as np
import matplotlib.cm as cm
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import matplotlib
from sklearn.metrics import silhouette_samples, silhouette_score
import nibabel as nib
from nilearn import plotting
from nilearn import image
import numpy as np
import os
import matplotlib.font_manager
plt.rcParams.update({'font.size': 16})


# Silhouette

In [ ]:
def get_cluster_id_to_distilled_searchlights(K, distill_numLights,
                      exaggerated_0, weight_it, cap_zero, fraction, threshold, by_sil):
    cluster_id_to_distilled_lights = {}
    for cluster_id in range(1,K+1):
        ranks_dir = "/scratch/gpfs/scollin/clustering_output/kmeans_searchlight_ranks/K" + str(K) + "/kmeans" + str(K) + "cluster" + str(cluster_id) + "_ranks"
        if by_sil:
            ranks_dir += "_by_silhouttes"
        else:
            if exaggerated_0:
                ranks_dir += ("_Consider0_T" + str(threshold))
            else:
                ranks_dir += "_NoConsider0"
            if weight_it:
                ranks_dir += ("_WeightByMean")
            else:
                ranks_dir += "_NoWeightByMean"
            if cap_zero:
                ranks_dir += "_CapZero"
            else:
                ranks_dir += "_NoCapZero"
        ranks_dir += ".csv"
        print(ranks_dir)
        ranks_df = pd.read_csv(ranks_dir)
        ranked_searchlights_in_cluster = ranks_df["searchlights_in_cluster"].tolist()
        if fraction:
            cluster_id_to_distilled_lights[cluster_id] = ranked_searchlights_in_cluster[0:int(distill_numLights*len(ranked_searchlights_in_cluster))]
        else: 
            cluster_id_to_distilled_lights[cluster_id] = ranked_searchlights_in_cluster[0:distill_numLights]

    return cluster_id_to_distilled_lights



In [ ]:
sil_avg_list = []
exaggerated_0 = False
weight_it = True
cap_zero = False
distill_it = False
distill_numLights = 0.2
fraction = True
threshold = 1.7
by_sil = False
matplotlib.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams['font.sans-serif'] =  "Arial"
for K in range(2,7):
    # get silhouttes
    in_dir = "/scratch/gpfs/scollin/clustering_output/kmeans_silhouette_tval/kmeans" + str(K) + "_silhouttes_tval.csv"
    labels_dir = "/scratch/gpfs/scollin/clustering_output/kmeans_assignments_tval/kmeans_" + str(K) + "clusters_tval.csv"
    df = pd.read_csv(labels_dir)
    cluster_labels = np.array(df["cluster_assignment"].tolist())
    searchlight_np = np.array((df["searchlight"].tolist()))
    sample_silhouette_values = np.loadtxt(in_dir, delimiter = ",")
    print(len(sample_silhouette_values))
    silhouette_avg = sum(sample_silhouette_values) / len(sample_silhouette_values)
    sil_avg_list.append(silhouette_avg)
    if distill_it:
        cluster_id_to_distilled_lights = get_cluster_id_to_distilled_searchlights(K, distill_numLights, exaggerated_0, weight_it, cap_zero, fraction, threshold, by_sil)
        all_lights_thisK = []
        for key in cluster_id_to_distilled_lights:
            for x in cluster_id_to_distilled_lights[key]:
                all_lights_thisK.append(x)
        filter_list = [True if light_id in all_lights_thisK else False for light_id in searchlight_np]
        searchlight_np = searchlight_np[filter_list]
        cluster_labels = cluster_labels[filter_list]
        sample_silhouette_values = sample_silhouette_values[filter_list]
    print(len(sample_silhouette_values))
    print(K, "clusters", ", Average silhoutte Score:", silhouette_avg)
    #plt.hist(sample_silhouette_values)
    n_clusters = K
    # plot them
    fig, (ax1) = plt.subplots(1, 1)
    fig.set_size_inches(25, 15)
    ax1.set_xlim([-0.1, 0.4])
    ax1.set_ylim([0, len(cluster_labels) + (n_clusters + 1) * 10])
    y_lower = 10
    for i in range(1,n_clusters + 1):
      
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        color = cm.nipy_spectral(1)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        y_lower = y_upper + 10  # 10 for the 0 samples
        ax1.set_title("K = " + str(i))
        ax1.set_xlabel("the silhouette coefficient values")
        ax1.set_ylabel("cluster label")
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks([-0.1, 0, 0.2])
        
    #changed by SHPC FEB 23
    save_name = "sil_K" + str(i) + ".png"
    plt.savefig(save_name, format = "png")
        
    plt.show()


In [ ]:
plt.plot([x for x in range(2,7)], sil_avg_list)
plt.title("silhoutte avg across K clusters in range [2,7]")


In [ ]:

def sort_list(ranks, lights):
    zipped_pairs = zip(ranks, lights)
    new_ranks = []
    new_lights = []
    for r,l in sorted(zipped_pairs, reverse= True):
        new_ranks.append(r)
        new_lights.append(l)
    return new_ranks, new_lights


# for each cluster, get the mean of that cluster
K = 5
labels_dir = "/scratch/gpfs/rk1593/clustering_output/kmeans_assignments_tval/kmeans_" + str(K) + "clusters_tval.csv"
df = pd.read_csv(labels_dir)
cluster_labels = df["cluster_assignment"]
searchlight_list = np.load("/scratch/gpfs/rk1593/clustering_output/kmeans_assignments_tval/master_searchlight_tval.npy")
X = np.load("/scratch/gpfs/rk1593/clustering_output/kmeans_assignments_tval/master_X_list_tval.npy")
silhouttes_dir = "/scratch/gpfs/rk1593/clustering_output/kmeans_silhouette_tval/kmeans" + str(K) + "_silhouttes_tval.csv"
searchlights_rank_list = np.loadtxt(silhouttes_dir, delimiter = ",")

for cluster_id in set(cluster_labels):
    searchlights_in_cluster = searchlight_list[cluster_labels == cluster_id]
    searchlights_in_cluster_ranks = searchlights_rank_list[cluster_labels == cluster_id]
    new_ranks, new_lights = sort_list(ranks = searchlights_in_cluster_ranks, lights = searchlights_in_cluster)
    output_dir = "/scratch/gpfs/rk1593/clustering_output/kmeans_searchlight_ranks/K" + str(K) + "/kmeans" + str(K) + "cluster" + str(cluster_id) + "_ranks_by_silhouttes.csv"
    df = pd.DataFrame({"searchlights_rank": new_ranks,
                       "searchlights_in_cluster":new_lights})
    df.to_csv(output_dir)

# Visualize data

In [ ]:

def get_cluster_id_to_distilled_searchlights(K, distill_numLights,
                      exaggerated_0, weight_it, cap_zero, fraction, threshold, by_sil):
    cluster_id_to_distilled_lights = {}
    for cluster_id in range(1,K+1):
        ranks_dir = "/scratch/gpfs/scollin/clustering_output/kmeans_searchlight_ranks/K" + str(K) + "/kmeans" + str(K) + "cluster" + str(cluster_id) + "_ranks"
        if by_sil:
            ranks_dir += "_by_silhouttes"
        else:
            if exaggerated_0:
                ranks_dir += ("_Consider0_T" + str(threshold)) 
            else:
                ranks_dir += "_NoConsider0"
            if weight_it:
                ranks_dir += ("_WeightByMean")
            else:
                ranks_dir += "_NoWeightByMean"
            if cap_zero:
                ranks_dir += "_CapZero"
            else:
                ranks_dir += "_NoCapZero"
        ranks_dir += ".csv"
        print(ranks_dir)
        ranks_df = pd.read_csv(ranks_dir)
        ranked_searchlights_in_cluster = ranks_df["searchlights_in_cluster"].tolist()
        if fraction:
            cluster_id_to_distilled_lights[cluster_id] = ranked_searchlights_in_cluster[0:int(distill_numLights*len(ranked_searchlights_in_cluster))]
        else: 
            cluster_id_to_distilled_lights[cluster_id] = ranked_searchlights_in_cluster[0:distill_numLights]

    return cluster_id_to_distilled_lights

def plot_colored_brain(K,cluster_assignment_csv_path, brain_image_path, optional_focus = None,
                               colored_brain_np = [], distill_it = False, distill_numLights = 1000,by_sil = True,
                      exaggerated_0 = False, weight_it = True,
                       cap_zero = False, fraction = False, threshold = None):
    """
    This function plots a colored brain based on the cluster assignment of kmodes

    cluster_assignment_csv_path: data frame with columns: "cluster_assignment", "searchlight", "features"

    brain_image_path: shape (97, 115, 97) image which will be used to get the affine and header

    output_path: where these plots will end up saved
    """
    # get the brain_image_path
    matplotlib.rcParams['font.family'] = "sans-serif"
    matplotlib.rcParams['font.sans-serif'] =  "Arial"
    img = nib.load(brain_image_path)
    if distill_it:
        cluster_id_to_distilled_lights = get_cluster_id_to_distilled_searchlights(K, distill_numLights, exaggerated_0, weight_it, cap_zero, fraction, threshold, by_sil)
    # get colored_brain_np if it is not fed in
    if len(colored_brain_np) == 0:
        # get the cluster assignment
        df = pd.read_csv(cluster_assignment_csv_path)
        colored_brain_np = np.zeros(shape=(97, 115, 97))
        # colored_brain_np = colored_brain_np.astype(float)
        # populate the brain_np with the cluster assignment
        for index,row in df.iterrows():
            light_id_splitted = row["searchlight"].split("_")
            light_id_str = row["searchlight"]
            cluster_id = int(row["cluster_assignment"])
            if distill_it and light_id_str not in cluster_id_to_distilled_lights[cluster_id]:
                continue
            x = int(light_id_splitted[0])
            y = int(light_id_splitted[1])
            z = int(light_id_splitted[2])
            colored_brain_np[x,y,z] = int(row["cluster_assignment"])
    original_colored_brain_np = copy.copy(colored_brain_np)
    if optional_focus != None:
        colored_brain_np[colored_brain_np != optional_focus] = 0
    # turn into nifti and plot with nilearn
    colored_brain = nib.Nifti1Image(colored_brain_np, affine=img.affine, header = img.header, extra = img.extra)
    print("optional_focus: ", optional_focus)
    print("len(np.unique(colored_brain_np)): ", len(np.unique(colored_brain_np)))
    print(np.unique(colored_brain_np))
    display = plotting.plot_roi(colored_brain, draw_cross = False, cut_coords = (-4,-30,20), colorbar = True, cmap = cm.get_cmap('Spectral', 20), title =  "" )#"K = " + str(K) + " & Focus Cluster = " + str(optional_focus) )
    plt.savefig("map5.pdf", format = "pdf")
    plt.show()#savefig("map.pdf", format = "pdf")
    display.close()
    # added by SHPC
    nib.save(colored_brain,"colored_brain_k2.nii")
    return original_colored_brain_np
    

# save brain map with all p values

In [ ]:
# settings
output_dir = "/scratch/gpfs/scollin/clustering_output/brainmap_pvals/each_searchlight/"

In [ ]:
# 
allSearchlightsIMG = "/home/scollin/wedding_schema/code/python/kmeans/notebooks/colored_brain_K5_allSearchlights.nii" 
img3 = nib.load(allSearchlightsIMG)
dataimg3 = img3.get_fdata()

result_schema = []

colored_brain_np_schema = np.zeros(shape=(97, 115, 97))
colored_brain_np_path = np.zeros(shape=(97, 115, 97))
colored_brain_np_perception = np.zeros(shape=(97, 115, 97))
colored_brain_np_rotated = np.zeros(shape=(97, 115, 97))

for filename in os.listdir('/scratch/gpfs/scollin/clustering_output/brainmap_pvals/each_searchlight/'):
    filename = os.path.join(filename)
    f = open(output_dir+filename, "r")
    fileContent = f.read()
    
    # convert string of file to dictionary for easier access
    res = json.loads(fileContent)
    
    resultValueSCHEMA = res['schema'] 
    resultValuePATH = res['path'] 
    resultValuePERCEPTION = res['perception'] 
    resultValueROTATED = res['rotated']
    
    # save x y z values of searchlight ID
    filename = filename.split("_")
    
    colored_brain_np_schema[int(filename[0]),int(filename[1]),int(filename[2])] = resultValueSCHEMA
    colored_brain_np_path[int(filename[0]),int(filename[1]),int(filename[2])] = resultValuePATH
    colored_brain_np_perception[int(filename[0]),int(filename[1]),int(filename[2])] = resultValuePERCEPTION
    colored_brain_np_rotated[int(filename[0]),int(filename[1]),int(filename[2])] = resultValueROTATED

brain_image_path = "/home/scollin/wedding_schema/code/python/kmeans/rentireAAL_wfupickatlas.nii" 
img = nib.load(brain_image_path)

colored_brain_schema = nib.Nifti1Image(colored_brain_np_schema, affine=img.affine, header = img.header, extra = img.extra)
nib.save(colored_brain_schema,"schema_pVals_all.nii")     

colored_brain_path = nib.Nifti1Image(colored_brain_np_path, affine=img.affine, header = img.header, extra = img.extra)
nib.save(colored_brain_path,"path_pVals_all.nii")   

colored_brain_perception = nib.Nifti1Image(colored_brain_np_perception, affine=img.affine, header = img.header, extra = img.extra)
nib.save(colored_brain_perception,"perception_pVals_all.nii")   

colored_brain_rotated = nib.Nifti1Image(colored_brain_np_rotated, affine=img.affine, header = img.header, extra = img.extra)
nib.save(colored_brain_rotated,"rotated_pVals_all.nii")   

# save fdr corrected fdr brain map

In [ ]:
from statsmodels.stats.multitest import multipletests

In [ ]:
allPvalsIMG = "/home/scollin/wedding_schema/code/python/kmeans/notebooks/schema_pVals_all.nii" 

brain_image_path = "/home/scollin/wedding_schema/code/python/kmeans/rentireAAL_wfupickatlas.nii" 
img = nib.load(brain_image_path)

img3 = nib.load(allPvalsIMG)
dataimg3 = img3.get_fdata()


In [ ]:
# set outside brain voxels to nans
dataimgmask= img.get_fdata()
outsideBrain_coords = np.where(dataimgmask==0)
dataimg3[outsideBrain_coords] = np.nan

In [ ]:
# Get voxels without NaNs
nonnan_mask = ~np.isnan(dataimg3)
nonnan_coords = np.where(nonnan_mask)

# Mask p-value map to exclude NaNs
nonnan_p = dataimg3[nonnan_mask]
#nonnan_data = dataimg3[nonnan_mask]

In [ ]:
# Get FDR-controlled q-values
nonnan_q = multipletests(nonnan_p, method='fdr_by')[1]
threshold = .05
print(f"{np.sum(dataimg3 < threshold)} significant voxels "
      f"at uncorrected p value of {threshold}")
print(f"{np.sum(nonnan_q < threshold)} significant voxels "
      f"controlling FDR at {threshold}")

# Threshold according FDR-controlled threshold
#nonnan_data[nonnan_q >= threshold] = np.nan

In [ ]:
# Reinsert thresholded back into whole brain image
p_thresh = np.full(dataimg3.shape, np.nan)
p_thresh[nonnan_coords] = nonnan_q

In [ ]:
p_thresh_image = nib.Nifti1Image(p_thresh, affine=img3.affine, header = img3.header)
nib.save(p_thresh_image,"schema_pVals_thresh_fdr0.05.nii") 

# Reinsert thresholded back into whole brain image but now INVERTED (1-correct p value)
p_thresh = np.full(dataimg3.shape, np.nan)
p_thresh[nonnan_coords] = 1-nonnan_q

p_thresh_image = nib.Nifti1Image(p_thresh, affine=img3.affine, header = img3.header)
nib.save(p_thresh_image,"schema_pVals_thresh_fdr0.05_inverted.nii")  

In [ ]:
## same for perception

allPvalsIMG = "/home/scollin/wedding_schema/code/python/kmeans/notebooks/perception_pVals_all.nii" 

img3 = nib.load(allPvalsIMG)
dataimg3 = img3.get_fdata()

# set outside brain voxels to nans
dataimg3[outsideBrain_coords] = np.nan

# Get voxels without NaNs
nonnan_mask = ~np.isnan(dataimg3)
nonnan_coords = np.where(nonnan_mask)

# Mask p-value map to exclude NaNs
nonnan_p = dataimg3[nonnan_mask]

# Get FDR-controlled q-values
nonnan_q = multipletests(nonnan_p, method='fdr_by')[1]
threshold = .05
print(f"{np.sum(dataimg3 < threshold)} significant voxels "
      f"at uncorrected p value of {threshold}")
print(f"{np.sum(nonnan_q < threshold)} significant voxels "
      f"controlling FDR at {threshold}")

# Reinsert thresholded back into whole brain image
p_thresh = np.full(dataimg3.shape, np.nan)
p_thresh[nonnan_coords] = nonnan_q

p_thresh_image = nib.Nifti1Image(p_thresh, affine=img3.affine, header = img3.header)
nib.save(p_thresh_image,"perception_pVals_thresh_fdr0.05.nii") 

# Reinsert thresholded back into whole brain image but now INVERTED (1-correct p value)
p_thresh = np.full(dataimg3.shape, np.nan)
p_thresh[nonnan_coords] = 1-nonnan_q

p_thresh_image = nib.Nifti1Image(p_thresh, affine=img3.affine, header = img3.header)
nib.save(p_thresh_image,"perception_pVals_thresh_fdr0.05_inverted.nii")   

In [ ]:
## same for path

allPvalsIMG = "/home/scollin/wedding_schema/code/python/kmeans/notebooks/path_pVals_all.nii" 

img3 = nib.load(allPvalsIMG)
dataimg3 = img3.get_fdata()

# set outside brain voxels to nans
dataimg3[outsideBrain_coords] = np.nan

# Get voxels without NaNs
nonnan_mask = ~np.isnan(dataimg3)
nonnan_coords = np.where(nonnan_mask)

# Mask p-value map to exclude NaNs
nonnan_p = dataimg3[nonnan_mask]

# Get FDR-controlled q-values
nonnan_q = multipletests(nonnan_p, method='fdr_by')[1]
threshold = .05
print(f"{np.sum(dataimg3 < threshold)} significant voxels "
      f"at uncorrected p value of {threshold}")
print(f"{np.sum(nonnan_q < threshold)} significant voxels "
      f"controlling FDR at {threshold}")

# Reinsert thresholded back into whole brain image
p_thresh = np.full(dataimg3.shape, np.nan)
p_thresh[nonnan_coords] = nonnan_q

p_thresh_image = nib.Nifti1Image(p_thresh, affine=img3.affine, header = img3.header)
nib.save(p_thresh_image,"path_pVals_thresh_fdr0.05.nii") 

# Reinsert thresholded back into whole brain image but now INVERTED (1-correct p value)
p_thresh = np.full(dataimg3.shape, np.nan)
p_thresh[nonnan_coords] = 1-nonnan_q

p_thresh_image = nib.Nifti1Image(p_thresh, affine=img3.affine, header = img3.header)
nib.save(p_thresh_image,"path_pVals_thresh_fdr0.05_inverted.nii")   

In [ ]:
## same for rotated

allPvalsIMG = "/home/scollin/wedding_schema/code/python/kmeans/notebooks/rotated_pVals_all.nii" 

img3 = nib.load(allPvalsIMG)
dataimg3 = img3.get_fdata()

# set outside brain voxels to nans
dataimg3[outsideBrain_coords] = np.nan

# Get voxels without NaNs
nonnan_mask = ~np.isnan(dataimg3)
nonnan_coords = np.where(nonnan_mask)

# Mask p-value map to exclude NaNs
nonnan_p = dataimg3[nonnan_mask]

# Get FDR-controlled q-values
nonnan_q = multipletests(nonnan_p, method='fdr_by')[1]
threshold = .05
print(f"{np.sum(dataimg3 < threshold)} significant voxels "
      f"at uncorrected p value of {threshold}")
print(f"{np.sum(nonnan_q < threshold)} significant voxels "
      f"controlling FDR at {threshold}")

# Reinsert thresholded back into whole brain image
p_thresh = np.full(dataimg3.shape, np.nan)
p_thresh[nonnan_coords] = nonnan_q

p_thresh_image = nib.Nifti1Image(p_thresh, affine=img3.affine, header = img3.header)
nib.save(p_thresh_image,"rotated_pVals_thresh_fdr0.05.nii") 

# Reinsert thresholded back into whole brain image but now INVERTED (1-correct p value)
p_thresh = np.full(dataimg3.shape, np.nan)
p_thresh[nonnan_coords] = 1-nonnan_q

p_thresh_image = nib.Nifti1Image(p_thresh, affine=img3.affine, header = img3.header)
nib.save(p_thresh_image,"rotated_pVals_thresh_fdr0.05_inverted.nii")   